# **Convolutional Neural Network - CNN**

## Objectives

* To classify the image set
* To test the performance of the classification using resizing vs resizing + padding

## Inputs

* CSVs outputted from ETL for resizing and resizing + padding

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

## Import Libraries

In [103]:
import numpy as np
import pandas as pd
import os
import base64
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, RandomFlip, RandomRotation, RandomZoom
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [104]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\Eddie\\Documents\\CodeInstitute Workspace\\Capstone Project'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [105]:
if os.path.basename(current_dir).lower() == "jupyter_notebooks":
    os.chdir(os.path.dirname(current_dir))
    print("You set a new current directory")

Confirm the new current directory

In [106]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\Eddie\\Documents\\CodeInstitute Workspace\\Capstone Project'

---

# Load CSV files

In [107]:
df_resize = pd.read_csv("./Data/Processed/df_resized.csv")
df_resize_padding = pd.read_csv("./Data/Processed/df_resized_padded.csv")

# Display the first few rows of each DataFrame
df_resize.head()
df_resize_padding.head()

,subfolder,luminance,contrast,image_path,grey_image_path
0,anger,202.218752,42.869499,./Data/Processed/Resized+Padded\image_0.png,./Data/Processed/Resized+Padded\grey_image_0.png
1,anger,134.196622,101.303135,./Data/Processed/Resized+Padded\image_1.png,./Data/Processed/Resized+Padded\grey_image_1.png
2,anger,109.144209,51.433215,./Data/Processed/Resized+Padded\image_2.png,./Data/Processed/Resized+Padded\grey_image_2.png
3,anger,140.261702,89.338355,./Data/Processed/Resized+Padded\image_3.png,./Data/Processed/Resized+Padded\grey_image_3.png
4,anger,143.852701,81.434234,./Data/Processed/Resized+Padded\image_4.png,./Data/Processed/Resized+Padded\grey_image_4.png


In [108]:
'''
#no longer using base64 encoding to store images in dataframes, instead storing file paths

#convert back to numpy arrays
def base64_to_cv2_array(b64_string):
    # Decode base64 → bytes
    img_bytes = base64.b64decode(b64_string)

    # Convert bytes → 1D uint8 array (image buffer)
    img_array = np.frombuffer(img_bytes, dtype=np.uint8)

    return cv2.imdecode(img_array, cv2.IMREAD_UNCHANGED)

df_resize['image'] = df_resize['image'].apply(lambda x: base64_to_cv2_array(x))
df_resize['grey_image'] = df_resize['grey_image'].apply(lambda x: base64_to_cv2_array(x))

df_resize_padding['image'] = df_resize_padding['image'].apply(lambda x: base64_to_cv2_array(x))
df_resize_padding['grey_image'] = df_resize_padding['grey_image'].apply(lambda x: base64_to_cv2_array(x))
'''

"\n#no longer using base64 encoding to store images in dataframes, instead storing file paths\n\n#convert back to numpy arrays\ndef base64_to_cv2_array(b64_string):\n    # Decode base64 → bytes\n    img_bytes = base64.b64decode(b64_string)\n\n    # Convert bytes → 1D uint8 array (image buffer)\n    img_array = np.frombuffer(img_bytes, dtype=np.uint8)\n\n    return cv2.imdecode(img_array, cv2.IMREAD_UNCHANGED)\n\ndf_resize['image'] = df_resize['image'].apply(lambda x: base64_to_cv2_array(x))\ndf_resize['grey_image'] = df_resize['grey_image'].apply(lambda x: base64_to_cv2_array(x))\n\ndf_resize_padding['image'] = df_resize_padding['image'].apply(lambda x: base64_to_cv2_array(x))\ndf_resize_padding['grey_image'] = df_resize_padding['grey_image'].apply(lambda x: base64_to_cv2_array(x))\n"

---

# CNN

Encode labels and split data into training, validation, and testing

In [109]:
#encode labels
encoder = LabelEncoder()
df_resize['encoded_labels'] = encoder.fit_transform(df_resize['subfolder'])
df_resize_padding['encoded_labels'] = encoder.fit_transform(df_resize_padding['subfolder'])

#split data into training, validation, and testing
train_df_resize, temp_df_resize = train_test_split(
    df_resize, test_size=0.4, random_state=3, stratify=df_resize['encoded_labels']
)
val_df_resize, test_df_resize = train_test_split(
    temp_df_resize, test_size=0.65, random_state=3, stratify=temp_df_resize['encoded_labels']
)

In [110]:
test_df_resize['encoded_labels'].value_counts()

encoded_labels
3    60
5    58
0    56
1    52
4    44
2    42
Name: count, dtype: int64

Colour images - resized

In [ ]:
data_augmentation = Sequential([
    RandomFlip("horizontal"),        # randomly flip left/right
    RandomRotation(0.1),             # rotate up to ±10%
    RandomZoom(0.1),                 # zoom in/out up to ±10%
])

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

'''
stacked_images = np.stack(train_df_resize['image'].values).astype('float16')/255.0
stacked_labels = np.stack(train_df_resize['encoded_labels'].values)
stacked_val_images = np.stack(val_df_resize['image'].values).astype('float16')/255.0
stacked_val_labels = np.stack(val_df_resize['encoded_labels'].values)
stacked_test_images = np.stack(test_df_resize['image'].values).astype('float16')/255.0
stacked_test_labels = np.stack(test_df_resize['encoded_labels'].values)
'''

import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def load_and_preprocess(path, label, augment=False):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)          # dtype uint8 [0..255]
    img = tf.cast(img, tf.float16) / 255.0              # normalize + convert per batch
    if augment:
        img = data_augmentation(img)
    return img, label

train_ds = tf.data.Dataset.from_tensor_slices((train_df_resize["image_path"], train_df_resize["encoded_labels"]))
train_ds = train_ds.map(lambda x, y: load_and_preprocess(x, y, augment=True), num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_df_resize["image_path"], val_df_resize["encoded_labels"]))
val_ds = val_ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_df_resize["image_path"], test_df_resize["encoded_labels"]))
test_ds = test_ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

#model.fit(stacked_images, stacked_labels, epochs=5, validation_data=(stacked_val_images, stacked_val_labels))

#test_loss, test_accuracy = model.evaluate(stacked_test_images, stacked_test_labels)

c:\Users\Eddie\Documents\CodeInstitute Workspace\Capstone Project\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [112]:
def inspect_arr(a, name):
    import numpy as np
    a = np.asarray(a)
    print(name, "dtype", a.dtype, "min", a.min(), "max", a.max(), "shape", a.shape)

# If using tf.data: fetch one batch and inspect
for x,y in train_ds.take(1):
    print("train batch", x.dtype, x.numpy().min(), x.numpy().max())

train batch <dtype: 'float32'> 0.0 1.0


In [113]:
model.fit(train_ds, epochs=10, validation_data=(val_ds))

test_loss, test_accuracy = model.evaluate(test_ds)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 17s 683ms/step - accuracy: 0.1944 - loss: 2.1368 - val_accuracy: 0.2143 - val_loss: 1.7589
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 648ms/step - accuracy: 0.2472 - loss: 1.7371 - val_accuracy: 0.2500 - val_loss: 1.7200
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 18s 755ms/step - accuracy: 0.3083 - loss: 1.7015 - val_accuracy: 0.2857 - val_loss: 1.7230
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 18s 640ms/step - accuracy: 0.3056 - loss: 1.7100 - val_accuracy: 0.2679 - val_loss: 1.7381
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 635ms/step - accuracy: 0.3389 - loss: 1.6503 - val_accuracy: 0.3452 - val_loss: 1.6773
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 624ms/step - accuracy: 0.3361 - loss: 1.6380 - val_accuracy: 0.3095 - val_loss: 1.6823
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 628ms/step - accuracy: 0.3667 - loss: 1.6031 - val_accuracy: 0.3095 - val_loss: 1.6927
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 633ms/step - accuracy: 0.4028 - loss: 1.5410 - val_accu

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.